# Phase 2: DeepSeek-OCR Accuracy Test

**Objective**: Test OCR accuracy on code-heavy conversation images.

**Target**: ≥90% character accuracy to proceed to Phase 3.

---

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → T4 GPU
2. **Run cells in order**: Don't skip cells
3. **Upload test image**: When prompted in Cell 3
4. **Document results**: Copy final output to `tests/PHASE2_RESULTS.md`

---

## Cell 1: Install Dependencies (~5 min)

In [ ]:
# Install required packages
!pip install -q transformers==4.46.3 tokenizers==0.20.3 einops pillow easydict addict
!pip install -q flash-attn --no-build-isolation

# Verify GPU availability
import torch
print("\n" + "="*70)
print(f"🎮 GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"🎮 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  WARNING: No GPU detected. This will be VERY slow.")
print("="*70)

## Cell 2: Download DeepSeek-OCR Model (~10 min, ~10GB)

In [ ]:
from huggingface_hub import snapshot_download
import os

print("Downloading DeepSeek-OCR model (~10GB)...")
print("This will take ~10 minutes on Colab's fast network.\n")

model_path = snapshot_download(
    repo_id='deepseek-ai/DeepSeek-OCR',
    local_dir='/content/deepseek-ocr',
    local_dir_use_symlinks=False
)

print(f"\n✓ Model downloaded to: {model_path}")
print(f"✓ Size: {sum(os.path.getsize(os.path.join(dirpath, filename)) for dirpath, dirnames, filenames in os.walk(model_path) for filename in filenames) / 1e9:.2f} GB")

## Cell 3: Upload Test Image (~5 sec)

**Action Required**: Upload `/tmp/poc_long_conversation.png` when prompted.

In [ ]:
from google.colab import files
import shutil

print("📤 Please upload: /tmp/poc_long_conversation.png\n")
uploaded = files.upload()

# Move to predictable path
image_filename = list(uploaded.keys())[0]
shutil.move(image_filename, '/content/test_image.png')

# Verify
from PIL import Image
img = Image.open('/content/test_image.png')
print(f"\n✓ Image ready at: /content/test_image.png")
print(f"✓ Dimensions: {img.width}x{img.height}")
print(f"✓ Format: {img.format}")
print(f"✓ Mode: {img.mode}")

## Cell 4: Define Original Text (instant)

This is the ground truth from `tests/poc_long_conversation.py:create_long_conversation()`

In [ ]:
# Original text from tests/poc_long_conversation.py (lines 22-168)
ORIGINAL_TEXT = """[USER]: I'm trying to implement a recursive function to find all files in a directory, but it's not working as expected. Here's my code:

import os

def find_files(path, pattern):
    results = []
    for item in os.listdir(path):
        if item.endswith(pattern):
            results.append(item)
    return results

files = find_files('/Users/me/Documents', '.txt')
print(files)

[ASSISTANT]: Your function is only checking the immediate directory, not recursing into subdirectories. Here's a fixed version:

import os

def find_files(path, pattern):
    results = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(pattern):
                full_path = os.path.join(root, file)
                results.append(full_path)
    return results

files = find_files('/Users/me/Documents', '.txt')
print(f'Found {len(files)} files:')
for f in files:
    print(f'  {f}')

The key changes:
1. Use os.walk() instead of os.listdir() - this recursively traverses directories
2. Store full paths using os.path.join()
3. Added better output formatting

[USER]: That works! But now I want to add a filter to exclude certain directories like __pycache__ and .git. How do I do that?

[ASSISTANT]: You can modify the dirs list in-place during os.walk() to skip directories:

import os

def find_files(path, pattern, exclude_dirs=None):
    if exclude_dirs is None:
        exclude_dirs = ['__pycache__', '.git', 'node_modules', '.venv']

    results = []
    for root, dirs, files in os.walk(path):
        # Remove excluded directories from dirs in-place
        dirs[:] = [d for d in dirs if d not in exclude_dirs]

        for file in files:
            if file.endswith(pattern):
                full_path = os.path.join(root, file)
                results.append(full_path)

    return results

# Usage
files = find_files('/Users/me/Documents', '.txt')
print(f'Found {len(files)} .txt files')

# Or with custom exclusions
files = find_files(
    '/Users/me/Projects',
    '.py',
    exclude_dirs=['build', 'dist', 'venv']
)

The key trick is `dirs[:] = [...]` which modifies the list in-place. This tells os.walk() to skip those directories.

[USER]: Perfect! One more thing - can I make it case-insensitive? I want to match both .txt and .TXT files.

[ASSISTANT]: Sure! Use str.lower() for case-insensitive matching:

import os

def find_files(path, pattern, exclude_dirs=None, case_sensitive=True):
    if exclude_dirs is None:
        exclude_dirs = ['__pycache__', '.git', 'node_modules', '.venv']

    # Normalize pattern for case-insensitive matching
    if not case_sensitive:
        pattern = pattern.lower()

    results = []
    for root, dirs, files in os.walk(path):
        dirs[:] = [d for d in dirs if d not in exclude_dirs]

        for file in files:
            # Compare lowercased filename if case-insensitive
            filename = file.lower() if not case_sensitive else file

            if filename.endswith(pattern):
                full_path = os.path.join(root, file)
                results.append(full_path)

    return results

# Case-insensitive search
files = find_files('/Users/me/Documents', '.txt', case_sensitive=False)
# Will match: document.txt, README.TXT, notes.Txt, etc.

[USER]: Excellent! This is exactly what I needed. Thanks!

[ASSISTANT]: You're welcome! A few best practices to consider:

1. **Error handling**: Add try-except for permission errors:
   ```python
   try:
       for root, dirs, files in os.walk(path):
           # ... your code ...
   except PermissionError as e:
       print(f"Permission denied: {e}")
   ```

2. **Performance**: For large directories, consider using pathlib:
   ```python
   from pathlib import Path

   def find_files_pathlib(path, pattern, case_sensitive=True):
       p = Path(path)
       if case_sensitive:
           return list(p.rglob(f'*{pattern}'))
       else:
           # pathlib doesn't support case-insensitive, use os.walk
           pass
   ```

3. **Testing**: Add unit tests:
   ```python
   def test_find_files():
       # Create temp directory structure
       import tempfile
       with tempfile.TemporaryDirectory() as tmpdir:
           # Create test files
           Path(tmpdir, 'test.txt').touch()
           Path(tmpdir, 'subdir').mkdir()
           Path(tmpdir, 'subdir', 'nested.txt').touch()

           # Test
           results = find_files(tmpdir, '.txt')
           assert len(results) == 2
   ```

Let me know if you need help with any of these!"""

print(f"Original text: {len(ORIGINAL_TEXT)} characters")
print(f"Estimated tokens: ~{len(ORIGINAL_TEXT) // 4}")

## Cell 5: Load Model & Run OCR (~10-30 sec inference)

This is the critical test. Expected latency on T4: 10-30 seconds.

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import time

print("Loading DeepSeek-OCR model...")
tokenizer = AutoTokenizer.from_pretrained(
    '/content/deepseek-ocr',
    trust_remote_code=True
)

model = AutoModel.from_pretrained(
    '/content/deepseek-ocr',
    _attn_implementation='flash_attention_2',
    trust_remote_code=True
).eval().cuda().to(torch.bfloat16)

print("✓ Model loaded to GPU\n")

# Run OCR
print("Running OCR inference...")
print("(This may take 10-30 seconds on T4 GPU)\n")

start = time.time()
result = model.infer(
    tokenizer,
    prompt="<image>\nFree OCR.",
    image_file='/content/test_image.png',
    base_size=1024,
    image_size=1024,
    crop_mode=False,
    save_results=False
)
elapsed = time.time() - start

print(f"✓ OCR complete in {elapsed:.2f}s")
print(f"✓ Output: {len(result)} characters")

## Cell 6: Calculate Accuracy & Display Results

Character-level accuracy using difflib.

In [ ]:
import difflib

# Calculate accuracy
matcher = difflib.SequenceMatcher(None, ORIGINAL_TEXT.strip(), result.strip())
accuracy = matcher.ratio() * 100

# Display results
print("\n" + "="*70)
print("🎯 PHASE 2 RESULTS")
print("="*70)
print(f"CHARACTER ACCURACY: {accuracy:.2f}%")
print(f"LATENCY: {elapsed:.2f}s")
print(f"ORIGINAL: {len(ORIGINAL_TEXT)} chars")
print(f"OCR OUTPUT: {len(result)} chars")
print(f"DIFFERENCE: {abs(len(result) - len(ORIGINAL_TEXT))} chars")
print("="*70)

# Token compression estimate
text_tokens = len(ORIGINAL_TEXT) // 4
visual_tokens = 256 * 2  # 2 pages at 1024x1024 = 256 tokens each
token_compression = text_tokens / visual_tokens
print(f"\nEst. text tokens: ~{text_tokens}")
print(f"Visual tokens (2 pages): {visual_tokens}")
print(f"Token compression: {token_compression:.2f}x")

## Cell 7: Show Comparisons

Visual comparison of original vs OCR output.

In [ ]:
# Show first 800 chars comparison
print("\n" + "="*70)
print("📄 ORIGINAL (first 800 chars):")
print("="*70)
print(ORIGINAL_TEXT[:800])

print("\n" + "="*70)
print("📄 OCR OUTPUT (first 800 chars):")
print("="*70)
print(result[:800])

# Show differences if accuracy < 100%
if accuracy < 100:
    print("\n" + "="*70)
    print("⚠️  SAMPLE DIFFERENCES (first 20):")
    print("="*70)
    
    diff_count = 0
    for i, s in enumerate(difflib.ndiff(ORIGINAL_TEXT[:500], result[:500])):
        if s[0] != ' ':
            print(f"Pos {i}: {repr(s)}")
            diff_count += 1
            if diff_count >= 20:
                print(f"... (showing first 20 of many differences)")
                break
else:
    print("\n✅ PERFECT MATCH: 100% accuracy!")

## Cell 8: Decision Tree

Based on accuracy, determine next steps.

In [ ]:
print("\n" + "="*70)
print("🎯 DECISION FOR PHASE 3")
print("="*70)

if accuracy >= 90:
    print("\n✅ SUCCESS: Accuracy ≥90%")
    print("\nNext Steps:")
    print("1. Document results in tests/PHASE2_RESULTS.md")
    print("2. Proceed to Phase 3: Full Implementation")
    print("   - Rewrite MCP server with OCR functions")
    print("   - Update client (rosie_client.py) for archive/reconstruct")
    print("   - Integrate with garbage collector")
    print("   - Write comprehensive tests")
    print("\n💡 Visual memory compression is VIABLE for production.")
    
elif accuracy >= 85:
    print("\n⚠️  MARGINAL: Accuracy 85-89%")
    print("\nNext Steps:")
    print("1. Iterate on rendering:")
    print("   - Try different fonts (DejaVu Sans Mono, Courier)")
    print("   - Adjust font size (12pt, 14pt, 16pt)")
    print("   - Test grayscale vs color")
    print("   - Disable syntax highlighting")
    print("2. Try different OCR prompts:")
    print("   - '<image>\\n<|grounding|>OCR this image.'")
    print("   - '<image>\\n<|grounding|>Convert to markdown.'")
    print("3. Re-test with adjustments")
    print("\n💡 Close, but needs optimization before production.")
    
else:
    print("\n❌ INSUFFICIENT: Accuracy <85%")
    print("\nNext Steps:")
    print("1. Document findings in tests/PHASE2_FINDINGS.md")
    print("2. Pivot to alternative approaches:")
    print("   Option A: Hybrid (PNG + text embeddings)")
    print("   Option B: Pure summarization (no visual compression)")
    print("   Option C: Skip visual memory entirely")
    print("3. Update project roadmap")
    print("\n💡 Visual memory compression NOT viable with current approach.")

print("\n" + "="*70)
print(f"📊 Summary: {accuracy:.2f}% accuracy in {elapsed:.2f}s")
print("="*70)